In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

In [3]:
# k
ks = [5, 10, 50]

score_function_dict = {
    "Recall"      : evaluators.recall
    #"Unpopularity": evaluators.unpopularity,
    #"Unpopularity2": evaluators.unpopularity2,
    #"Unpopularity3": evaluators.unpopularity3,
    #"F1-score"    : evaluators.f1_score,
    #"F1-score2"    : evaluators.f1_score2,
    #"F1-score3"    : evaluators.f1_score3
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)
# coverage = evaluators.CoverageEvaluator(test_set, ks)
# hubness = evaluators.HubnessEvaluator(test_set, ks)

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [10]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
valid_per_epoch = 50
n_item_sample = 30
n_user_sample = 30
no_progressbar = False

search_range = 30

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma, n_item_sample, n_user_sample, bias=0.1)
mymp = searches.Mymp(model, search_range)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [11]:
trainer.fit(n_batch, n_epoch)

epoch50 avg_loss:0.238: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:03<00:00, 75.39it/s]


In [12]:
trainer.valid(knn, userwise)
re1 = trainer.valid_scores.copy()
display(re1)
trainer.valid(mp, userwise)
re2 = trainer.valid_scores.copy()
display(re2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:04<00:00, 193.59it/s]


,Recall@5,Recall@10,Recall@50
0,0.404032,0.570325,0.889178


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:12<00:00, 72.46it/s]


,Recall@5,Recall@10,Recall@50
0,0.264889,0.408442,0.787347


In [18]:
mp = searches.MutualProximity(model, gamma, n_item_sample, n_user_sample, bias=1.0)
trainer.valid(mp, userwise)
re2 = trainer.valid_scores.copy()
display(re2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:23<00:00, 40.54it/s]


,Recall@5,Recall@10,Recall@50
0,0.403279,0.570156,0.889178


In [ ]:
mp2 = searches.MutualProximity(model, gaussian)

In [ ]:
trainer.valid(mp, recall)

In [ ]:
trainer.valid_scores